# algorithms for massive data - market-basket analysis on amazon books reviews

## libraries import

In [146]:
import os
import zipfile
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, size, udf, array_contains, explode, row_number
from pyspark.sql.types import ArrayType, StringType
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover, NGram
from pyspark.ml.fpm import FPGrowth
import nltk
from nltk.stem import WordNetLemmatizer
from pyspark.sql.window import Window
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import time
import pandas as pd
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [147]:
# NLTK downloads for lemmatization
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

## general parameters

In [148]:
SAMPLE_SIZE = 0.01 # dataset sample fraction
SEED = 42 # random seed
MIN_SUPPORT = 0.01 # minumum support threshold for frequent itemset
MIN_CONFIDENCE = 0.2 # minimum confidence threshold for association rules

In [149]:
# words to keep in the analysis and used for obtaining meaningful bigrams.
# basically here we have words that will be removed from the most frequent words
# that are going to be searched automatically and removed. also these words are usefull because
# a bigram will be considered into our final basket if and only if one of the 2 words
# in the bigram is contained into this list.

KEEP_WORDS = {"good", "great", "excellent", "amazing", "wonderful", "brilliant", "fantastic",
              "bad", "terrible", "awful", "disappointing", "boring", "dull", "most",
              "love", "like", "hate", "best", "worst", "favorite", "recommended",
              "well", "interesting", "better", "easy", "recommend", "must", "high",
              "highly","very", "really", "quite", "much", "many", "lot", "enjoy", "prefer",
}

In [150]:
# words to be removed. include common stopwors and terms that could lead to insignificant
# words associations like 'higly' comporting 'recommended'.
# this list is a second iteration after the automatic stopwords removal based on frequency

BAN_TOKENS = {
    "take", "point", "last", "school", "young", "year", "get", "even", "something",
    "way", "make", "all", "just", "being", "over", "both", "through", "yourselves", "its",
    "before", "herself", "had", "should", "to", "only", "under", "ours", "has", "tell",
    "do", "them", "his", "they", "not", "during", "now", "him", "nor", "writing", "seem",
    "did", "this", "she", "each", "further", "where", "few", "because", "doing", "become",
    "some", "are", "our", "ourselves", "out", "what", "for", "while", "does", "look",
    "above", "between", "t", "be", "we", "who", "were", "here", "hers", "by", "use",
    "on", "about", "of", "against", "s", "or", "own", "into", "yourself", "down", "try",
    "your", "from", "her", "their", "there", "been", "whom", "too", "themselves",
    "was", "until", "more", "himself", "that", "but", "don", "with", "written", "leave",
    "those", "he", "me", "myself", "these", "will", "below", "can", "going", "may",
    "theirs", "my", "and", "then", "is", "am", "it", "an", "as", "itself", "at", "keep",
    "have", "in", "any", "if", "again", "no", "when", "how", "other", "also", "go",
    "which", "you", "after", "such", "why", "a", "off", "i", "yours", "always",
    "so", "the", "having", "once", "ago", "know", "want", "think", "find", "thing",
    "ever", "two", "end", "call", "first", "page", "found", "than", "another",
    "work", "see", "give", "every", "come", "say", "put", "still", "would",
    "day", "back", "made", "though", "however", "read", "later", "one", "need"
}

## data imports

In [151]:
os.environ['KAGGLE_USERNAME'] = "antdepa"
os.environ['KAGGLE_KEY'] = "bb4586df4d1661596f68edd3d214b0c0"

In [152]:
zip_path = "./data/amazon-books-reviews.zip"
csv_path = "./data/Books_rating.csv"

In [153]:
# dataset download if not preset

if not os.path.exists(zip_path) and not os.path.exists(csv_path):
    os.system("kaggle datasets download -d mohamedbakhet/amazon-books-reviews -p ./data")

In [154]:
# extraction of the csv format dataset from the zip file if needed
if not os.path.exists(csv_path):
    with zipfile.ZipFile(zip_path, 'r') as z:
        for file in z.namelist():
            if "Books_rating.csv" in file:
                z.extract(file, "./data")
                os.rename(f"./data/{file}", csv_path)

## spark session initialization

In [155]:
spark = SparkSession.builder.appName("Frequent_items") \
    .master("local[*]") \
    .getOrCreate()

In [156]:
# sampled dataset

books = spark.read.csv(csv_path, header=True, inferSchema=True).sample(SAMPLE_SIZE,SEED)

In [157]:
books.show(10)

+----------+--------------------+-----+--------------+--------------------+------------------+------------+-----------+--------------------+--------------------+
|        Id|               Title|Price|       User_id|         profileName|review/helpfulness|review/score|review/time|      review/summary|         review/text|
+----------+--------------------+-----+--------------+--------------------+------------------+------------+-----------+--------------------+--------------------+
|0312322291|King James: Belie...| NULL|          NULL|                NULL|               2/2|         5.0| 1118188800|Yo Homie this is ...|"LeBron James is ...|
|B0006D6DRK|Open marriage;: A...| NULL|A3KBF2S2MGN48O|"T. Thompson ""&#...|               2/2|         4.0| 1188604800|A Must Read for a...|This book is a cl...|
|0671551345|Night World: Daug...| NULL|          NULL|                NULL|               0/0|         5.0|  889920000|The most charming...|The plot and char...|
|B0008852GG|The soul of man 

In [158]:
books.count()

30332

## null values handling

In [159]:
# keep only reviews where the content is not null and not just empty whitespace

books_clean = books.filter(
    (~col('review/text').isNull()) &  # condition: 'review/text' must NOT be null
    (~col('review/text').rlike(r'^\s*$'))  # condition: 'review/text' must NOT be empty or only whitespace
)

In [160]:
books_clean.show(10)

+----------+--------------------+-----+--------------+--------------------+------------------+------------+-----------+--------------------+--------------------+
|        Id|               Title|Price|       User_id|         profileName|review/helpfulness|review/score|review/time|      review/summary|         review/text|
+----------+--------------------+-----+--------------+--------------------+------------------+------------+-----------+--------------------+--------------------+
|0312322291|King James: Belie...| NULL|          NULL|                NULL|               2/2|         5.0| 1118188800|Yo Homie this is ...|"LeBron James is ...|
|B0006D6DRK|Open marriage;: A...| NULL|A3KBF2S2MGN48O|"T. Thompson ""&#...|               2/2|         4.0| 1188604800|A Must Read for a...|This book is a cl...|
|0671551345|Night World: Daug...| NULL|          NULL|                NULL|               0/0|         5.0|  889920000|The most charming...|The plot and char...|
|B0008852GG|The soul of man 

## tokenization

In [161]:
# tokenization of the text by non-word characters and later convert it to lowercase

tokenizer = RegexTokenizer(
    inputCol="review/text",
    outputCol="tokens",
    pattern=r"\W+", # split on non-word characters
    toLowercase=True
)

In [162]:
tokenized = tokenizer.transform(books_clean)

## lemmatization

In [163]:
lemmatizer = WordNetLemmatizer()

In [164]:
def lemmatize_tokens(tokens):
    if not tokens:
        return []
    return [lemmatizer.lemmatize(t) for t in tokens]

In [165]:
lemmatize_udf = udf(lemmatize_tokens, ArrayType(StringType()))

In [166]:
tokens_lemmatized = tokenized.withColumn("tokens_lem", lemmatize_udf(col("tokens")))

## stopwords

In [167]:
stopwords_remover = StopWordsRemover(
    inputCol="tokens_lem",
    outputCol="tokens_nostopwords"
)

In [168]:
no_stopwords = stopwords_remover.transform(tokens_lemmatized)

## frequent words to be removed

In [169]:
def get_top_frequent_words(spark_df, col_name="tokens_nostopwords", top_n=20):
    freqs = (spark_df
             .select(explode(col(col_name)).alias("token"))
             .groupBy("token")
             .count()
             .orderBy(col("count").desc())
             .limit(top_n))
    return [row['token'] for row in freqs.collect()]

In [170]:
auto_stopwords = set(get_top_frequent_words(no_stopwords, "tokens_nostopwords", top_n=50))

In [171]:
print("Stopwords filtered:", sorted(list(auto_stopwords)))

Stopwords filtered: ['also', 'author', 'best', 'book', 'character', 'doe', 'even', 'find', 'first', 'found', 'get', 'go', 'good', 'great', 'ha', 'know', 'life', 'like', 'little', 'love', 'm', 'make', 'many', 'much', 'never', 'new', 'novel', 'one', 'page', 'people', 'quot', 'read', 'reader', 'reading', 'really', 'see', 'story', 'take', 'thing', 'think', 'time', 'two', 'wa', 'want', 'way', 'well', 'work', 'world', 'written', 'year']


In [172]:
auto_stopwords = auto_stopwords - KEEP_WORDS # remove words that are explicitly KEEP_WORDS

In [173]:
print("Stopwords filtered:", sorted(list(auto_stopwords)))

Stopwords filtered: ['also', 'author', 'book', 'character', 'doe', 'even', 'find', 'first', 'found', 'get', 'go', 'ha', 'know', 'life', 'little', 'm', 'make', 'never', 'new', 'novel', 'one', 'page', 'people', 'quot', 'read', 'reader', 'reading', 'see', 'story', 'take', 'thing', 'think', 'time', 'two', 'wa', 'want', 'way', 'work', 'world', 'written', 'year']


In [174]:
auto_stopwords = auto_stopwords - {"author", "character", "novel", "story"}

In [175]:
def filter_tokens(tokens):
    if not tokens:
        return [] # return [] if input is empty

    words_to_remove = (auto_stopwords | BAN_TOKENS) - KEEP_WORDS

    # keep tokens that are:
    # - not in words_to_remove
    # - longer than 2 chars
    # - not purely digits
    return [t for t in tokens
            if t not in words_to_remove
            and len(t) > 2
            and not t.isdigit()]


In [176]:
filter_udf = udf(filter_tokens, ArrayType(StringType()))

In [177]:
tokens_filtered = no_stopwords.withColumn("tokens_filtered", filter_udf(col("tokens_nostopwords")))

## bigrams

In [178]:
bigram = NGram(n=2, inputCol="tokens_filtered", outputCol="bigrams_raw")

In [179]:
with_bigrams = bigram.transform(tokens_filtered)

In [180]:
# return only bigrams that start with a negative word, joined by '_'

def filter_neg_bigrams(bigrams):
    if not bigrams or not isinstance(bigrams, list):
        return []

    NEGATIVE_STARTERS = {"didnt", "dont", "never", "cannot", "no", "not", "wont", "isnt", "wasnt", "couldnt", "shouldnt"}
    result = []

    for bg in bigrams:
        w1, w2 = bg.split()
        if w1 in NEGATIVE_STARTERS:
            result.append(f"{w1}_{w2}")

    return result

In [181]:
filter_didnt_bigrams_udf = udf(filter_neg_bigrams, ArrayType(StringType()))

In [182]:
with_bigrams = with_bigrams.withColumn("meaningful_bigrams", filter_didnt_bigrams_udf(col("bigrams_raw")))

## tokens and bigrams combination

In [183]:
def combine_tokens_bigrams(tokens, bigrams):
    tokens = tokens or []
    bigrams = bigrams or []
    return list(dict.fromkeys(tokens + bigrams))

In [184]:
combine_udf = udf(combine_tokens_bigrams, ArrayType(StringType()))

In [185]:
combined = with_bigrams.withColumn("basket", combine_udf(col("tokens_filtered"), col("meaningful_bigrams")))

## sample baskets

In [186]:
combined.show(20, truncate=True)

+----------+--------------------+-----+--------------+--------------------+------------------+------------+-----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------+--------------------+
|        Id|               Title|Price|       User_id|         profileName|review/helpfulness|review/score|review/time|      review/summary|         review/text|              tokens|          tokens_lem|  tokens_nostopwords|     tokens_filtered|         bigrams_raw|meaningful_bigrams|              basket|
+----------+--------------------+-----+--------------+--------------------+------------------+------------+-----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------+--------------------+
|0312322291|King James: Belie...| NULL|          NULL|                NULL|    

In [187]:
sample_baskets = combined.take(100)
all_items = []
for basket in sample_baskets:
    all_items.extend(basket['basket'])

bigram_items = [item for item in all_items if '_' in item]
single_items = [item for item in all_items if '_' not in item]

print(f"Bigrams found: {len(set(bigram_items))}")
print(f"Single token: {len(set(single_items))}")
print(f"Bigram examples: {sorted(set(bigram_items))[:5]}")

Bigrams found: 3
Single token: 2201
Bigram examples: ['didnt_anything', 'didnt_like', 'didnt_mind']


## FPGrowth model for frequent itemsets

In [188]:
fpGrowth = FPGrowth(itemsCol="basket", minSupport=MIN_SUPPORT, minConfidence=MIN_CONFIDENCE)

In [189]:
model = fpGrowth.fit(combined)

In [ ]:
# display frequent itemsets, association rules, and predictions

print("Frequent Itemsets:")
model.freqItemsets.orderBy("freq", ascending=False).show(truncate=False)

print("Association Rules:")
model.associationRules.orderBy("confidence", ascending=False).show(truncate=False)

print("Predictions:")
predictions = model.transform(combined)
predictions.orderBy("prediction", ascending=False).show(5, truncate=False)

Frequent Itemsets:


In [ ]:
model.associationRules.filter((col("lift") > 1.2) & (col("confidence") > 0.3)).show(truncate=False) # filtering rules with strong lift and confidence

In [ ]:
windowSpec = Window.orderBy(col("confidence").desc())

In [ ]:
rules_with_index = model.associationRules.withColumn("row_num", row_number().over(windowSpec))

In [ ]:
rules_with_index.filter(col("row_num") > 20).show(truncate=False)

In [ ]:
rules_with_index.filter(col("row_num") > 40).show(truncate=False)

In [ ]:
rules_with_index.filter(col("row_num") > 60).show(truncate=False)

## visualizations

In [ ]:
freq_itemsets = model.freqItemsets.toPandas()
assoc_rules = model.associationRules.toPandas()

In [ ]:
top_items = freq_itemsets.sort_values(by="freq", ascending=False).head(20) # top 20 frequent itemsets
top_items['items_str'] = top_items['items'].apply(lambda x: ', '.join(x))

In [ ]:
plt.figure(figsize=(12,6))
sns.barplot(x="freq", y="items_str", data=top_items, palette="summer")
plt.title("Top 20 Frequent Itemsets")
plt.xlabel("Frequency")
plt.ylabel("Itemset")
plt.show()

In [ ]:
top_rules = assoc_rules.sort_values(by="confidence", ascending=False).head(20) # top 20 association rules
top_rules['rule_str'] = top_rules['antecedent'].apply(lambda x: ', '.join(x)) + " -> " + top_rules['consequent'].apply(lambda x: ', '.join(x))

In [ ]:
plt.figure(figsize=(12,6))
sns.barplot(x="confidence", y="rule_str", data=top_rules, palette="summer")
plt.title("Top 20 Association Rules")
plt.xlabel("Confidence")
plt.ylabel("Rule")
plt.show()

## parameter sensitivity analysis

In [ ]:
# performs a grid search over different minSupport and minConfidence values

def parameter_sensitivity_analysis(combined_df):
    support_values = [0.005, 0.01, 0.02]
    confidence_values = [0.1, 0.2, 0.3]
    results = []

    for support in support_values:
        for confidence in confidence_values:
            print(f"\nTesting: Support={support}, Confidence={confidence}")
            start = time.time()

            # fit the model with the given support and confidence
            model = FPGrowth(itemsCol="basket", minSupport=support, minConfidence=confidence).fit(combined_df)

            # count frequent itemsets and association rules
            num_itemsets = model.freqItemsets.count()
            num_rules = model.associationRules.count()

            # measure time
            final_time = time.time() - start

            results.append({
                "support": support,
                "confidence": confidence,
                "num_itemsets": num_itemsets,
                "num_rules": num_rules,
                "training_time": final_time
            })

            print(f"Itemsets: {num_itemsets:,}")
            print(f"Rules: {num_rules:,}")
            print(f"Time: {final_time:.2f}s")

    return pd.DataFrame(results)

In [ ]:
param_results = parameter_sensitivity_analysis(combined)
print("\n-------------------summary table-------------------")
print(param_results.to_string(index=False))

In [ ]:
# heatmaps for sensitivity analysis

fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# heatmap 1: number of frequent itemsets for each support-confidence combination
pivot_itemsets = param_results.pivot(index='support', columns='confidence', values='num_itemsets')
sns.heatmap(pivot_itemsets, annot=True, fmt='d', ax=axes[0,0], cmap='summer')
axes[0,0].set_title('Number of Frequent Itemsets')

# heatmap 2: number of association rules for each support-confidence combination
pivot_rules = param_results.pivot(index='support', columns='confidence', values='num_rules')
sns.heatmap(pivot_rules, annot=True, fmt='d', ax=axes[0,1], cmap='summer')
axes[0,1].set_title('Number of Association Rules')

# heatmap 3: training time in seconds for each parameter combination
pivot_time = param_results.pivot(index='support', columns='confidence', values='training_time')
sns.heatmap(pivot_time, annot=True, fmt='.2f', ax=axes[1,0], cmap='summer')
axes[1,0].set_title('Training Time (seconds)')

# heatmap 4: ratio of rules per itemset for each combination
param_results['rules_per_itemset'] = param_results['num_rules'] / param_results['num_itemsets']
pivot_ratio = param_results.pivot(index='support', columns='confidence', values='rules_per_itemset')
sns.heatmap(pivot_ratio, annot=True, fmt='.2f', ax=axes[1,1], cmap='summer')
axes[1,1].set_title('Rules per Itemset Ratio')

plt.tight_layout()
plt.show()